# Today's cryptocurrencies historical data on by Market Cap 

In [19]:
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import pendulum
from sqlalchemy import create_engine
from webdriver_manager.chrome import ChromeDriverManager
import numpy as np
from splinter import Browser
from bs4 import BeautifulSoup
import os
import requests
import json

#### Context : Finance

#### Goal: Our goal is to create tables showing today’s 10 best/most profitable cryptocurrencies to invest in. We will look at historical data for the previous year and create a database by hourly and daily price changes. 

#### METHOD: we aim to do this by extracting data based on the ranks of the cryptocurrency and also the daily and hourly change of the cryptocurrency. 

#### FINDING DATA: Following are the sources for finding and collecting our data. Sources: Cryptocurrencies: https://coinmarketcap.com/ 
CSV files: https://www.cryptodatadownload.com/data/binance/
APIs: https://financialmodelingprep.com/api/v3/ https://rest.coinapi.io/v1/

#### Data collection tools: Splinter APIs BeautifulSoup

#### DATA AND CLEANUP: 
##### Cleanup process: Pandas SqlAlchemy Database: PostgreSQL

##### TEAM MEMBERS Juan Castaneda Elif Evrim Polat Nichole Edet

<div><h3 style="color:green;">Web Scraping - Obtaining today's best cryptocurrencies liste at https://coinmarketcap.com </h3></div>

In [2]:
# Obtaining Tables at the next link using pandas
url = 'https://coinmarketcap.com/'

In [3]:
tables = pd.read_html(url)

In [4]:
df = tables[0]
df.head()

,Unnamed: 0,#,Name,Price,24h %,7d %,Market Cap,Volume(24h),Circulating Supply,Last 7 Days,Unnamed: 10
0,NaN,1.0,Bitcoin1BTCBuy,"$62,812.71",0.64%,11.62%,"$1,173,509,467,382","$76,740,651,4911,221,738 BTC","18,682,675 BTC",NaN,NaN
1,NaN,2.0,Ethereum2ETHBuy,"$2,418.15",5.18%,21.55%,"$279,222,954,046","$34,554,547,54814,289,669 ETH","115,469,714 ETH",NaN,NaN
2,NaN,3.0,Binance Coin3BNBBuy,$541.66,3.09%,43.96%,"$83,704,505,133","$7,376,658,55413,618,569 BNB","154,532,785 BNB",NaN,NaN
3,NaN,4.0,XRP4XRP,$1.80,0.84%,98.86%,"$81,691,352,234","$29,348,569,13116,311,925,766 XRP","45,404,028,640 XRP",NaN,NaN
4,NaN,5.0,Cardano5ADA,$1.46,3.90%,23.53%,"$46,753,895,562","$9,542,541,6626,520,698,868 ADA","31,948,309,441 ADA",NaN,NaN


In [5]:
# using only the tables that will match our PostgreSQL database schema
columns = ['#', 'Name', 'Price']
cryptos_df = df[columns]
cryptos_df.head()

,#,Name,Price
0,1.0,Bitcoin1BTCBuy,"$62,812.71"
1,2.0,Ethereum2ETHBuy,"$2,418.15"
2,3.0,Binance Coin3BNBBuy,$541.66
3,4.0,XRP4XRP,$1.80
4,5.0,Cardano5ADA,$1.46


In [6]:
# converting # column values to integer
cryptos_df['#'] = cryptos_df['#'].astype(float).map("{:.0f}".format).copy()
cryptos_df.rename(columns={'#':'id','Name':'coin_name','Price':'latest_price'}, inplace=True)
# obtaiting only the first 10 coins
cryptos_df = cryptos_df[:10]

In [7]:
cryptos_df.head()

,id,coin_name,latest_price
0,1,Bitcoin1BTCBuy,"$62,812.71"
1,2,Ethereum2ETHBuy,"$2,418.15"
2,3,Binance Coin3BNBBuy,$541.66
3,4,XRP4XRP,$1.80
4,5,Cardano5ADA,$1.46


In [8]:
# Splitting Name column into 2 different columns which will containt the coin name and coin symbol using a number as a delimiter
# and using n=1 to do this only once, in case the coin name contains a number already
columnsplit = cryptos_df['coin_name'].str.split('(\d+)',n=1, expand=True)
columnsplit.head()

,0,1,2
0,Bitcoin,1,BTCBuy
1,Ethereum,2,ETHBuy
2,Binance Coin,3,BNBBuy
3,XRP,4,XRP
4,Cardano,5,ADA


###### Formatting table to match our postgreSQL table in our DB

In [9]:
columnsplit = cryptos_df['coin_name'].str.split('(\d+)',n=1, expand=True)
# since coin_symbol column originally contains a recommendation to "Buy" if applicable on coinmarketcap, we will get rid of that
# columnsplit[1] = columnsplit[1].str.replace('Buy', '', regex=True) by ignoring index 1
cryptos_df = cryptos_df.assign(coin_name=columnsplit[0],coin_symbol=columnsplit[2].str.replace('Buy',''))

In [10]:
cryptos_df

,id,coin_name,latest_price,coin_symbol
0,1,Bitcoin,"$62,812.71",BTC
1,2,Ethereum,"$2,418.15",ETH
2,3,Binance Coin,$541.66,BNB
3,4,XRP,$1.80,XRP
4,5,Cardano,$1.46,ADA
5,6,Tether,$1.00,USDT
6,7,Polkadot,$42.18,DOT
7,8,Uniswap,$35.28,UNI
8,9,Litecoin,$276.41,LTC
9,10,Chainlink,$40.02,LINK


In [11]:
# rearranging columns
columns = cryptos_df.columns.to_list()

In [12]:
# new order
columns = columns[:2] + [columns[-1]] + [columns[-2]]
columns

['id', 'coin_name', 'coin_symbol', 'latest_price']

In [13]:
cryptos_df = cryptos_df[columns]
cryptos_df

,id,coin_name,coin_symbol,latest_price
0,1,Bitcoin,BTC,"$62,812.71"
1,2,Ethereum,ETH,"$2,418.15"
2,3,Binance Coin,BNB,$541.66
3,4,XRP,XRP,$1.80
4,5,Cardano,ADA,$1.46
5,6,Tether,USDT,$1.00
6,7,Polkadot,DOT,$42.18
7,8,Uniswap,UNI,$35.28
8,9,Litecoin,LTC,$276.41
9,10,Chainlink,LINK,$40.02


<div><h3 style="color:green;">Dowloading the CSV files if available at https://www.cryptodatadownload.com/data/binance/</h3></div>

In [16]:
# Obtaining symbols in the dataframe and storing in a list
symbols = cryptos_df['coin_symbol'].to_list()
symbols

['BTC', 'ETH', 'BNB', 'XRP', 'ADA', 'USDT', 'DOT', 'UNI', 'LTC', 'LINK']

###### Web Scraping using beautiulsoup

In [21]:
# URL of page to be scraped
url = 'https://www.cryptodatadownload.com/data/binance/'

# instantiating the webdriver for Chrome!!!
browser.visit(url)
# Getting the webpage content
html = browser.html
# parsing our html plain text to a BS object
soup = BeautifulSoup(html, 'html.parser')
browser.quit()

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 89.0.4389
[WDM] - Get LATEST driver version for 89.0.4389
[WDM] - Driver [C:\Users\jich-\.wdm\drivers\chromedriver\win32\89.0.4389.23\chromedriver.exe] found in cache
